# Загрузка модели и подготовка сабмита

In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader

В зависимости от модели в этой клетке будут разные импорты

In [2]:
import sys
sys.path.append(os.path.expanduser('/home/jovyan/ocp-airi/ocpmodels/models'))
sys.path.append(os.path.expanduser('/home/jovyan/ocp-airi/'))

from spinconv import spinconv

Для каждой модели надо написать свою фукнцию preproceccing(), но обязательно, чтобы в массиве keys был элемент 'sid'

In [3]:
def preprocessing(system):
    keys = ['sid', 'pos', 'atomic_numbers', 'cell', 'natoms']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

Функция, которая расчитывает сабмит

In [4]:
def inference(model, iterator):
    y = torch.tensor([])
    model.eval()

    with torch.no_grad():
        for index, data_list in enumerate(tqdm(iterator)):
            sids = torch.tensor([[data.sid] for data in data_list])
            predictions = model(data_list).squeeze()
            predictions = predictions.reshape((predictions.shape[0], 1)) 

            mini_submit = torch.cat((sids, predictions.to('cpu')), dim=1)
            y = torch.cat((y, mini_submit))

    return y

In [6]:
batch_size = 4
num_workers = -1

target_col = 'y_relaxed'
features_cols = ['feature_1']

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


Здесь мы задаем адрес чекпоинта модели, надо вставить адрес чекпоинта

In [17]:
# адрес датасета
test_dataset_path = "../../ocp_datasets/data/is2re_test_challenge_2021/data.lmdb"
# адрес чекпоинта модели
model_adress = "/home/jovyan/ocp-airi/airi_utils/2021-09-25-11-23-25_epoch_15_.pickle"

В этой клетке надо раскомментить первую строчку и закомментить вторую

In [18]:
train_dataset_file_path= os.path.expanduser(test_dataset_path)

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

В этой клетке надо раскомментить первую строчку и закомментить вторую

In [10]:
# если модель сохраняется через state_dict, то процедура загрузки модели отличается
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
model = torch.load(model_adress)
# model = spinconv(None, None, 1, otf_graph=True, regress_forces=False)

#переносим на куду если она есть
model = model.to(device)

In [11]:
%%time

# print(f'Start training model {str(model)}')
submission = inference(model, training_generator)

  0%|          | 0/2500 [00:00<?, ?it/s]

ml-test-server-0:18580:18580 [0] NCCL INFO Bootstrap : Using [0]eth0:10.233.122.86<0>
ml-test-server-0:18580:18580 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
ml-test-server-0:18580:18580 [0] NCCL INFO NET/IB : No device found.
ml-test-server-0:18580:18580 [0] NCCL INFO NET/Socket : Using [0]eth0:10.233.122.86<0>
ml-test-server-0:18580:18580 [0] NCCL INFO Using network Socket
NCCL version 2.7.8+cuda10.2
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 00/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 01/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 02/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 03/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 04/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 05/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 06/12 :    0   1
ml-test-server-0:18580:18603 [0] NCCL INFO Channel 07/12 :    0   1
ml

100%|██████████| 2500/2500 [11:56<00:00,  3.49it/s]

CPU times: user 3min 25s, sys: 43.8 s, total: 4min 9s
Wall time: 11min 56s


In [13]:
# submission = np.random.randn(2, 5)
submission

tensor([[2472718.0000,     -0.5258],
        [1747243.0000,     -1.2762],
        [1824821.0000,     -0.0377],
        ...,
        [1314291.0000,     -1.3288],
        [1972741.0000,     -3.7513],
        [1189586.0000,     -3.9547]])

In [14]:
np.savez("submission.npz", challenge_ids=submission[:, 0].int(), challenge_energy=submission[:, 1])

In [15]:
sub = np.load("submission.npz")
sub['challenge_ids']

array([2472718, 1747243, 1824821, ..., 1314291, 1972741, 1189586],
      dtype=int32)

In [16]:
sub['challenge_energy']

array([-0.5258029 , -1.2762375 , -0.03772031, ..., -1.3288329 ,
       -3.7513304 , -3.9547446 ], dtype=float32)